In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager
fablib = FablibManager(project_id="")# update project ID


In [ ]:
# --- variables you must define before building the slice ---
site1 = "UCSD"                # VM site
site2 = "UCSD"                # P4 switch site
slice_name = "INStofino"
node1_name = "Node1"
node2_name = "Node2"
p4_name    = "P4"
network1_name = "net1"
network2_name = "net2"
model = "NIC_Basic"           # keep this unless you specifically need ConnectX


In [ ]:
# Create Slice
slice = fablib.new_slice(name=slice_name)

# Create Networks (note: both use 192.168.0.0/24 in the tutorial)
net1 = slice.add_l2network(name=network1_name)
net2 = slice.add_l2network(name=network2_name)
 
# Node 1 @ site1 on net1
node1 = slice.add_node(name=node1_name, site=site1)
iface1 = node1.add_component(model=model, name='nic1').get_interfaces()[0]
iface1.set_mode('config')
net1.add_interface(iface1)

# P4 switch @ site2 with two ports: iface2 on net1, iface3 on net2
p4 = slice.add_switch(name=p4_name, site=site2)
iface2 = p4.get_interfaces()[0]
iface3 = p4.get_interfaces()[1]
net1.add_interface(iface2)
net2.add_interface(iface3)

# Node 2 @ site1 on net2
node2 = slice.add_node(name=node2_name, site=site1)
iface4 = node2.add_component(model=model, name='nic1').get_interfaces()[0]
iface4.set_mode('config')
net2.add_interface(iface4)

# Submit
slice.submit()


In [ ]:
p4 = slice.get_node("P4") 
stdout, stderr = p4.execute(command=[
    # List all SDE shells on the box
    ("compgen -c | grep -E '^sde-env-' || true", r".*\$ ", 10),

    # Enter the newest one available
    ("if command -v sde-env-9.14.0 >/dev/null 2>&1; then "
     "  sde-env-9.14.0; "
     "elif command -v sde-env-9.13.3 >/dev/null 2>&1; then "
     "  sde-env-9.13.3; "
     "else "
     "  echo 'NO_SDE_FOUND'; "
     "fi",
     # prompt example: [nix-shell(SDE-9.13.3):~]$  or similar
     r"\[nix-shell\(SDE-[0-9.]+\):.*\]\$ ", 10),

    # Prove the toolchain is visible
    ("which bf-p4c || echo 'bf-p4c not in PATH'", r"\[nix-shell\(SDE-[0-9.]+\):.*\]\$ ", 10),

    # Leave the env
    ("exit", r".*\$ ", 10),
])